# Data Cleansing

In [15]:
import pandas as pd
import geopy.geocoders
from geopy.geocoders import Nominatim
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geopandas as gpd
import os
import numpy as np
from math import floor
from multiprocess import Pool, cpu_count
import os
import re
import pandas as pd

In [16]:
parent_path = "/Users/krishna/Rutgers/DataScience/Project/final_project/csv_files"

In [17]:
nypd_path = 'nypd_11_19.csv'
nypd_dataset = pd.read_csv(os.path.join(parent_path, nypd_path))

/var/folders/3k/c7cxyyxd19b76xmw693g1ftc0000gn/T/ipykernel_97603/3120013169.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  nypd_dataset = pd.read_csv(os.path.join(parent_path, nypd_path))


In [18]:
file_loc = "ny/ny.shp"
fp = os.path.join(parent_path, file_loc)
data = gpd.read_file(fp)

In [19]:
max = nypd_dataset.shape[0]
lim = round(max/cpu_count())
data_range = []
for i in range(0,cpu_count()):
    if i == (cpu_count()-1):
        data_range.append((lim*i, max))
    else:
        data_range.append((lim*i, lim*(i+1)))

# Responsible for identifying the zipcode using the longitude and latitude

In [20]:
def get_zip_code(data_range):
    zipcodes = []
    for index, rows in nypd_dataset.iterrows():
        is_found = False
        if index < data_range[0]:
            continue
        if index > data_range[1] - 1:
            break

        point = Point((rows['Longitude'], rows['Latitude']))
        for itr, geoms in enumerate(data['geometry']):
            if geoms.intersects(point):
                zipcodes.append(data["ZCTA5CE20"][itr])
                is_found = True
                break

        
        if not is_found:
            zipcodes.append(0)
    
    return zipcodes

In [21]:
zip_codes = []
with Pool(processes=cpu_count()) as p:
    for data in p.map(get_zip_code, data_range):
        zip_codes.extend(data)
        # zipcodes = get_zip_code(10000, 40000)

In [22]:
print(len(zip_codes), nypd_dataset.shape[0])

2589426 2589426


In [23]:
nypd_df = nypd_dataset

In [24]:
nypd_df["zipcode"] = zip_codes

In [26]:
nypd_df = nypd_df[nypd_df.zipcode != 0]

In [27]:
nypd_df

,Unnamed: 0,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,...,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,cmplt_year,zipcode
0,1,81715726,11/14/2011,20:15:00,NaN,(null),32,11/14/2011,578,HARRASSMENT 2,...,40.811255,-73.942946,"(40.811255, -73.942946)",PATROL BORO MAN NORTH,(null),45-64,BLACK,F,2011,10027
1,2,80795025,09/19/2011,10:30:00,NaN,(null),88,09/19/2011,105,ROBBERY,...,40.693265,-73.973593,"(40.693265, -73.973593)",PATROL BORO BKLYN NORTH,(null),<18,WHITE,M,2011,11205
2,3,80831000,09/20/2011,15:40:00,NaN,(null),70,09/20/2011,105,ROBBERY,...,40.630367,-73.973713,"(40.63036667, -73.97371311)",PATROL BORO BKLYN SOUTH,(null),<18,BLACK,M,2011,11230
3,5,80495855,08/27/2011,01:00:00,NaN,(null),28,08/27/2011,109,GRAND LARCENY,...,40.799978,-73.956584,"(40.799978, -73.956584)",PATROL BORO MAN NORTH,(null),25-44,WHITE,F,2011,10026
4,6,78975812,05/25/2011,15:06:00,NaN,(null),25,05/25/2011,106,FELONY ASSAULT,...,40.809214,-73.938109,"(40.809214, -73.938109)",PATROL BORO MAN NORTH,(null),25-44,BLACK,M,2011,10037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2589421,4340641,278999565,01/01/2013,00:01:00,12/31/2016,23:59:00,113,12/13/2023,104,RAPE,...,40.679981,-73.776234,"(40.6799807384666, -73.7762339071953)",PATROL BORO QUEENS SOUTH,(null),25-44,BLACK,F,2013,11434
2589422,4340642,274042435,12/27/2018,00:00:00,02/12/2019,23:14:00,101,09/07/2023,112,THEFT-FRAUD,...,40.596003,-73.742115,"(40.596003, -73.742115)",PATROL BORO QUEENS SOUTH,(null),45-64,BLACK,M,2018,11691
2589423,4340644,265348339,04/15/2014,16:00:00,12/15/2016,16:00:00,101,03/19/2023,233,SEX CRIMES,...,40.602776,-73.750117,"(40.6027762300781, -73.7501173648331)",PATROL BORO QUEENS SOUTH,(null),<18,BLACK,F,2014,11691
2589424,4340645,265919393,05/01/2018,00:00:00,05/14/2018,00:00:00,105,03/30/2023,104,RAPE,...,40.726529,-73.734865,"(40.7265293769835, -73.7348645378882)",PATROL BORO QUEENS SOUTH,(null),25-44,BLACK,F,2018,11428


In [28]:
nypd_df.drop(nypd_df.columns[nypd_df.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)

/var/folders/3k/c7cxyyxd19b76xmw693g1ftc0000gn/T/ipykernel_97603/3640588842.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nypd_df.drop(nypd_df.columns[nypd_df.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)


In [29]:
path = "nypd_final.csv"
nypd_df.to_csv(os.path.join(parent_path, path))

## NYC Populus Deatils
Details for the populus spread across multiple sheets are joint in this.

In [30]:
files_df = list()

parent_path = "/Users/krishna/Rutgers/DataScience/Project/final_project/csv_files"
child_path = ["income", "pop_employment", "schooling"]
for child in child_path:
    path = os.path.join(parent_path, child)
    for filename in os.listdir(path):
        if re.match(".*Data.csv", filename):
            year = filename.split(".", 1)[0][-4:]
            file_df = pd.read_csv(os.path.join(path, filename))[1:]
            name_zipcodes = []
            for val in file_df["NAME"]:
                name_zipcodes.append(val[-5:])

            file_df["year"] = year
            file_df["zipcode"] = name_zipcodes
            files_df.append(file_df)
    
    final_df = pd.concat(files_df)
    output_path = os.path.join(parent_path, child + ".csv")
    final_df.drop(final_df.columns[final_df.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)
    final_df.to_csv(output_path)
    files_df.clear()